In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
path = str(Path(Path('File.py').parent.absolute()).parent.absolute())
sys.path.insert(0, path)

path = str(Path(Path('thermo.py').parent.absolute()).parent.absolute())
sys.path.insert(0, path)

import numpy as np
import matplotlib as mlab
import matplotlib.pyplot as plt
import scipy
import xarray as xr
import os as os
import glob
import matplotlib.gridspec as gridspec
from tools.cbook import pickle2Obj, interp_z, compute_thetae
import tools
from tools.FV3_tools import read_solo_fields
from tools.CM1_tools import read_cm1_fields

from pltbook import nice_clevels, nice_mxmnintvl, container, plot_contour_row

import matplotlib.colors as mcolors

cmap_data = [(1.0, 1.0, 1.0),
         (0.3137255012989044, 0.8156862854957581, 0.8156862854957581),
         (0.0, 1.0, 1.0),
         (0.0, 0.8784313797950745, 0.501960813999176),
         (0.0, 0.7529411911964417, 0.0),
         (0.501960813999176, 0.8784313797950745, 0.0),
         (1.0, 1.0, 0.0),
         (1.0, 0.6274510025978088, 0.0),
         (1.0, 0.0, 0.0),
         (1.0, 0.125490203499794, 0.501960813999176),
         (0.9411764740943909, 0.250980406999588, 1.0),
         (0.501960813999176, 0.125490203499794, 1.0),
         (0.250980406999588, 0.250980406999588, 1.0),
         (0.125490203499794, 0.125490203499794, 0.501960813999176),
         (0.125490203499794, 0.125490203499794, 0.125490203499794),
         (0.501960813999176, 0.501960813999176, 0.501960813999176),
         (0.8784313797950745, 0.8784313797950745, 0.8784313797950745),
         (0.9333333373069763, 0.8313725590705872, 0.7372549176216125),
         (0.8549019694328308, 0.6509804129600525, 0.47058823704719543),
         (0.6274510025978088, 0.42352941632270813, 0.23529411852359772),
         (0.4000000059604645, 0.20000000298023224, 0.0)]

_nthreads = 2

# draw filled contours.
wclevs = [0, 1, 2.5, 5, 7.5, 10, 15, 20, 30, 40, 50, 60, 70]

tclevs = list(-40.0 + 2*np.arange(41))

teclevs = list(300. + 5*np.arange(70))

bclevs = list(-0.4+ 0.02*np.arange(100))
pclevs = list(-15.0 + np.arange(31))

dclevs = list(-20.0 + np.arange(41))

zlevels = 25.0 + 50.*np.arange(300)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
paths = {}
paths['cm1_3km_hdd125'] = '/work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00'
#paths['cm1_2km_hdd125'] = '/work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_test_hdd125_C2000_00'
#paths['cm1_1km_hdd125'] = '/work/wicker/Odin_scr/cm1r20.3/run/bubble_1km_dt01_hdd125_C2000_00'

cm1 = []

for key in paths:
    print("Reading: ",key)
    cm1.append( read_cm1_fields(paths[key], vars = ['den'], file_pattern=None, ret_dbz=False, ret_ds=False) )

Reading:  cm1_3km_hdd125
------------------------------------------------------------------------------------------------------------------------ 

 Added default filename to path input:  /work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00/cm1out.nc
------------------------------------------------------------------------------------------------------------------------ 

 Reading:  /work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00/cm1out.nc 

 Completed reading in:  /work/wicker/Odin_scr/cm1r20.3/run/bubble_3km_TESTA_hdd125_C2000_00/cm1out.nc 

------------------------------------------------------------------------------------------------------------------------


In [3]:
cm1[0]['zc'].shape[1]


60

In [4]:
import netCDF4 as ncdf

In [51]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass
ncfile = ncdf.Dataset('test3D_data.nc',mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)

x_dim = ncfile.createDimension('nx', cm1[0]['xc'].size)     
y_dim = ncfile.createDimension('ny', cm1[0]['yc'].size)   
z_dim = ncfile.createDimension('nz', cm1[0]['zc'].shape[1]) 
t_dim = ncfile.createDimension('time', None)

for dim in ncfile.dimensions.items():
    print(dim)
x = ncfile.createVariable('xh', np.float32, ('nx',))
x.units = 'meters'

y = ncfile.createVariable('yh', np.float32, ('ny',))
y.units = 'meters'

z = ncfile.createVariable('zh', np.float32, ('time','nz','ny','nx'))
z.units = 'meters'

t = ncfile.createVariable('time', np.float32, ('time',))
t.units = 'sec'

den = ncfile.createVariable('den', np.float32, ('time','nz','ny','nx'))
den.units = 'kg/m^3'

# write coordinates and data

x[:] = cm1[0]['xc'][:]
y[:] = cm1[0]['yc'][:]
t[:] = cm1[0]['sec'][:]

for n in np.arange(cm1[0]['den'].shape[0]):

    den[n,:,:,:] = cm1[0]['den'][n,:,:,:]
    z[n,:,:,:]   = cm1[0]['zc'][n,:,:,:]

print(ncfile)
ncfile.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
('nx', <class 'netCDF4._netCDF4.Dimension'>: name = 'nx', size = 64)
('ny', <class 'netCDF4._netCDF4.Dimension'>: name = 'ny', size = 64)
('nz', <class 'netCDF4._netCDF4.Dimension'>: name = 'nz', size = 60)
('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0)
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): nx(64), ny(64), nz(60), time(61)
    variables(dimensions): float32 xh(nx), float32 yh(ny), float32 zh(time, nz, ny, nx), float32 time(time), float32 den(time, nz, ny, nx)
    groups: 


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): nx(64), ny(64), nz(60), time(0)
    variables(dimensions): float32 xh(nx), float32 yh(ny), float32 zh(nz), float32 time(time), float32 den(nz, ny, nx)
    groups: 


In [56]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass
ncfile = ncdf.Dataset('test3D_data.nc',mode='r',format='NETCDF4_CLASSIC') 
print(ncfile)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): nx(64), ny(64), nz(60), time(61)
    variables(dimensions): float32 xh(nx), float32 yh(ny), float32 zh(time, nz, ny, nx), float32 time(time), float32 den(time, nz, ny, nx)
    groups: 


In [57]:
den = ncfile.variables['den']
print(den[0,0,0])
print(ncfile.variables['xh'][:])
print(ncfile.variables['zh'].shape)

[1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734 1.1602734
 1.1602734]
[  1.5000001   4.5         7.5000005  10.500001   13.500001   16.5
  19.5        22.500002   25.500002   28.500002   31.500002   34.5
  37.5        40.500004   43.500004   46.500004   49.500004   52.500004
  55.500004   58.500004   61.500004   64.5        67.5        70.5
  73.5        76.5        79.5        82.50001    85.50001    88.50001
  91.

In [58]:
ncfile.close()